<a href="https://colab.research.google.com/github/bcdanl/320-code/blob/main/danl_320_script_2025_0217.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CW 7

In [22]:
from pyspark.sql.functions import min, max, mean, col, avg, std

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, max, mean, col, avg, sd
from pyspark.sql.window import Window

spark = SparkSession.builder.master("local[*]").getOrCreate()
df_pd = pd.read_csv('https://bcdanl.github.io/data/cereal_oatmeal.csv')
cereal = spark.createDataFrame(df_pd)
cereal.show()

+--------------------+--------------+----+--------+-----+------+
|                Name|  Manufacturer|Type|Calories|Fiber|Sugars|
+--------------------+--------------+----+--------+-----+------+
|           100% Bran|       Nabisco|Cold|      70| 10.0|     6|
|   100% Natural Bran|   Quaker Oats|Cold|     120|  2.0|     8|
|            All-Bran|     Kellogg's|Cold|      70|  9.0|     5|
|All-Bran with Ext...|     Kellogg's|Cold|      50| 14.0|     0|
|      Almond Delight|Ralston Purina|Cold|     110|  1.0|     8|
|Apple Cinnamon Ch...| General Mills|Cold|     110|  1.5|    10|
|         Apple Jacks|     Kellogg's|Cold|     110|  1.0|    14|
|             Basic 4| General Mills|Cold|     130|  2.0|     8|
|           Bran Chex|Ralston Purina|Cold|      90|  4.0|     6|
|         Bran Flakes|          Post|Cold|      90|  5.0|     5|
|        Cap'n'Crunch|   Quaker Oats|Cold|     120|  0.0|    12|
|            Cheerios| General Mills|Cold|     110|  2.0|     1|
|Cinnamon Toast Cr...| Ge

In [3]:
cereal.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Manufacturer: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Calories: long (nullable = true)
 |-- Fiber: double (nullable = true)
 |-- Sugars: long (nullable = true)



In [5]:
cereal.describe().show(truncate = False)

+-------+-------------------+---------------------------+----+------------------+------------------+------------------+
|summary|Name               |Manufacturer               |Type|Calories          |Fiber             |Sugars            |
+-------+-------------------+---------------------------+----+------------------+------------------+------------------+
|count  |77                 |77                         |77  |77                |77                |77                |
|mean   |NULL               |NULL                       |NULL|106.88311688311688|2.1519480519480516|6.922077922077922 |
|stddev |NULL               |NULL                       |NULL|19.48411905682084 |2.3833639643872226|4.4448853924193585|
|min    |100% Bran          |American Home Food Products|Cold|50                |0.0               |-1                |
|max    |Wheaties Honey Gold|Ralston Purina             |Hot |160               |14.0              |15                |
+-------+-------------------+-----------

## Q1
Group the cereal DataFrame, using the Manufacturer variable.



In [6]:
grouped_df = cereal.groupBy("Manufacturer")
grouped_df

GroupedData[grouping expressions: [Manufacturer], value: [Name: string, Manufacturer: string ... 4 more fields], type: GroupBy]

## Q2
Determine the total number of groups, and the number of cereals per group.



In [7]:
(
    cereal
    .select("Manufacturer")
    .distinct()
    .count()
)

7

In [10]:
grouped_df.count().show(truncate = False)

+---------------------------+-----+
|Manufacturer               |count|
+---------------------------+-----+
|General Mills              |22   |
|Kellogg's                  |23   |
|Nabisco                    |6    |
|Ralston Purina             |8    |
|Quaker Oats                |8    |
|Post                       |9    |
|American Home Food Products|1    |
+---------------------------+-----+



## Q3
Extract the cereals that belong to the manufacturer "Kellogg's".



In [14]:
q3 = cereal.filter(col("Manufacturer") == "Kellogg's")
q3.show(n = q3.count(),
        truncate=False)

+---------------------------+------------+----+--------+-----+------+
|Name                       |Manufacturer|Type|Calories|Fiber|Sugars|
+---------------------------+------------+----+--------+-----+------+
|All-Bran                   |Kellogg's   |Cold|70      |9.0  |5     |
|All-Bran with Extra Fiber  |Kellogg's   |Cold|50      |14.0 |0     |
|Apple Jacks                |Kellogg's   |Cold|110     |1.0  |14    |
|Corn Flakes                |Kellogg's   |Cold|100     |1.0  |2     |
|Corn Pops                  |Kellogg's   |Cold|110     |1.0  |12    |
|Cracklin' Oat Bran         |Kellogg's   |Cold|110     |4.0  |7     |
|Crispix                    |Kellogg's   |Cold|110     |1.0  |3     |
|Froot Loops                |Kellogg's   |Cold|110     |1.0  |13    |
|Frosted Flakes             |Kellogg's   |Cold|110     |1.0  |11    |
|Frosted Mini-Wheats        |Kellogg's   |Cold|100     |3.0  |7     |
|Fruitful Bran              |Kellogg's   |Cold|120     |5.0  |12    |
|Just Right Crunchy 

## Q4
Calculate the average of values in the Calories, Fiber, and Sugars variables for each manufacturer.



In [15]:
cereal.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Manufacturer: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Calories: long (nullable = true)
 |-- Fiber: double (nullable = true)
 |-- Sugars: long (nullable = true)



In [16]:
(
    grouped_df
    .agg(
        avg("Calories").alias("mean_Calories"),
        avg("Fiber").alias("mean_Fiber"),
        avg("Sugars").alias("mean_Sugars")
    ).show()
)

+--------------------+------------------+------------------+------------------+
|        Manufacturer|     mean_Calories|        mean_Fiber|       mean_Sugars|
+--------------------+------------------+------------------+------------------+
|       General Mills|111.36363636363636|1.2727272727272727| 7.954545454545454|
|           Kellogg's|108.69565217391305| 2.739130434782609| 7.565217391304348|
|             Nabisco| 86.66666666666667|               4.0|1.8333333333333333|
|      Ralston Purina|             115.0|             1.875|             6.125|
|         Quaker Oats|              95.0|            1.3375|              5.25|
|                Post|108.88888888888889|2.7777777777777777| 8.777777777777779|
|American Home Foo...|             100.0|               0.0|               3.0|
+--------------------+------------------+------------------+------------------+



## Q5
Find the maximum value in the Sugars variable for each manufacturer.



In [19]:
grouped_df.max("Sugars").show()

+--------------------+-----------+
|        Manufacturer|max(Sugars)|
+--------------------+-----------+
|       General Mills|         14|
|           Kellogg's|         15|
|             Nabisco|          6|
|      Ralston Purina|         11|
|         Quaker Oats|         12|
|                Post|         15|
|American Home Foo...|          3|
+--------------------+-----------+



## Q6
Find the minimum value in the Fiber variable for each manufacturer.



In [20]:
grouped_df.min("Fiber").show()

+--------------------+----------+
|        Manufacturer|min(Fiber)|
+--------------------+----------+
|       General Mills|       0.0|
|           Kellogg's|       0.0|
|             Nabisco|       1.0|
|      Ralston Purina|       0.0|
|         Quaker Oats|       0.0|
|                Post|       0.0|
|American Home Foo...|       0.0|
+--------------------+----------+



## Q7
Calculate a ‘Normalized_Sugars’ variable for each product by Manufacturer, where the normalization formula is

$$
\text{Normalized\_Sugars} = \frac{\text{Sugars} - \text{mean(Sugars)}}{\text{std(Sugars)}}
$$

for each Manufacturer group. This formula adjusts the sugar content of each product by subtracting the mean sugar content of its manufacturer and then dividing by the standard deviation of the sugar content within its manufacturer.

In [23]:
# Define a window partitioned by "Type"
w = Window.partitionBy("Manufacturer")

df_with_standardized = cereal.withColumn(
    "normalized_sugar_by_company",
    (col("Sugars") - mean(col("Sugars")).over(w) ) / std(col("Sugars")).over(w)
)
df_with_standardized.show()

+--------------------+--------------------+----+--------+-----+------+---------------------------+
|                Name|        Manufacturer|Type|Calories|Fiber|Sugars|normalized_sugar_by_company|
+--------------------+--------------------+----+--------+-----+------+---------------------------+
|               Maypo|American Home Foo...| Hot|     100|  0.0|     3|                       NULL|
|Apple Cinnamon Ch...|       General Mills|Cold|     110|  1.5|    10|         0.5281722017019429|
|             Basic 4|       General Mills|Cold|     130|  2.0|     8|       0.011737160037821054|
|            Cheerios|       General Mills|Cold|     110|  2.0|     1|        -1.7957854857866051|
|Cinnamon Toast Cr...|       General Mills|Cold|     120|  0.0|     9|        0.26995468086988195|
|            Clusters|       General Mills|Cold|     110|  2.0|     7|       -0.24648036079423985|
|         Cocoa Puffs|       General Mills|Cold|     110|  0.0|    13|         1.3028247641981254|
|       Co